In [35]:
#dependencies
import pandas as pd

In [36]:
#loading file
all_disney_movies = 'Resources/disney_movies.csv'

In [37]:
#testing file
read_file = pd.read_csv(all_disney_movies)
read_file

,movie_title,release_date,genre,mpaa_rating,total_gross,inflation_adjusted_gross
0,Snow White and the Seven Dwarfs,1937-12-21,Musical,G,184925485,5228953251
1,Pinocchio,1940-02-09,Adventure,G,84300000,2188229052
2,Fantasia,1940-11-13,Musical,G,83320000,2187090808
3,Song of the South,1946-11-12,Adventure,G,65000000,1078510579
4,Cinderella,1950-02-15,Drama,G,85000000,920608730
...,...,...,...,...,...,...
574,The Light Between Oceans,2016-09-02,Drama,PG-13,12545979,12545979
575,Queen of Katwe,2016-09-23,Drama,PG,8874389,8874389
576,Doctor Strange,2016-11-04,Adventure,PG-13,232532923,232532923
577,Moana,2016-11-23,Adventure,PG,246082029,246082029


In [38]:
# retrieve year and month
read_file['release_date'] = pd.to_datetime(read_file.release_date)
year = read_file['release_date'].dt.strftime("%Y")
month = read_file['release_date'].dt.strftime("%m")

In [39]:
# add year and month columns
read_file["Release Year"] = year
read_file["Release Month"] = month
read_file.head()

,movie_title,release_date,genre,mpaa_rating,total_gross,inflation_adjusted_gross,Release Year,Release Month
0,Snow White and the Seven Dwarfs,1937-12-21,Musical,G,184925485,5228953251,1937,12
1,Pinocchio,1940-02-09,Adventure,G,84300000,2188229052,1940,02
2,Fantasia,1940-11-13,Musical,G,83320000,2187090808,1940,11
3,Song of the South,1946-11-12,Adventure,G,65000000,1078510579,1946,11
4,Cinderella,1950-02-15,Drama,G,85000000,920608730,1950,02


In [50]:
# filter data
clean_file = read_file.dropna(how = 'any')
clean_file = clean_file.loc[clean_file["Release Year"] >= '1990']
clean_file = clean_file.rename(columns={"movie_title":"Movie Title", "release_date":"Release Date", "genre":"Genre", "mpaa_rating":"MPAA Rating", "inflation_adjusted_gross":"Inflation-Adjusted Gross"})
clean_file = clean_file.drop(columns=["total_gross"])
clean_file = clean_file[["Movie Title", "Release Date", "Release Year", "Release Month", "Genre", "MPAA Rating", "Inflation-Adjusted Gross"]]
clean_file

,Movie Title,Release Date,Release Year,Release Month,Genre,MPAA Rating,Inflation-Adjusted Gross
85,Stella,1990-02-02,1990,02,Drama,PG-13,40077147
87,Pretty Woman,1990-03-23,1990,03,Romantic Comedy,R,356389765
88,Ernest Goes to Jail,1990-04-06,1990,04,Comedy,PG,49999822
89,Spaced Invaders,1990-04-27,1990,04,Comedy,PG,29964452
90,Fire Birds,1990-05-24,1990,05,Action,PG-13,29485923
...,...,...,...,...,...,...,...
574,The Light Between Oceans,2016-09-02,2016,09,Drama,PG-13,12545979
575,Queen of Katwe,2016-09-23,2016,09,Drama,PG,8874389
576,Doctor Strange,2016-11-04,2016,11,Adventure,PG-13,232532923
577,Moana,2016-11-23,2016,11,Adventure,PG,246082029
